In [4]:
from models.ConvE import ConvE, ConvE_args
from utils.loaders import load_data
import pickle
import torch
import numpy as np
from sklearn.utils import shuffle

In [5]:
args = ConvE_args()

In [6]:
with open(r'C:\Users\odhra\Documents\graph_project\allosteric_reasonings\data\processed\triplets.pickle', 'rb') as handle:
    data = pickle.load(handle)

with open(r'C:\Users\odhra\Documents\graph_project\allosteric_reasonings\data\processed\triplet_lookup.pickle', 'rb') as handle:
    lookup = pickle.load(handle)

with open(r'C:\Users\odhra\Documents\graph_project\allosteric_reasonings\data\processed\ASD_dictionary.pickle', 'rb') as handle:
    ASD_dictionary = pickle.load(handle)


with open(r'C:\Users\odhra\Documents\graph_project\allosteric_reasonings\data\processed\Sparse_dictionary.pickle', 'rb') as handle:
    BCE_dictionary = pickle.load(handle)


entities = int(len(lookup)/2)

In [7]:
number_of_batches = 10
number_of_epochs = 30
x = shuffle(data)
test_data = x[:100] #just limit data to the first 100 rows

In [8]:
all_objects, all_relationships, all_subjects = np.split(test_data, 3, axis=1)
all_objects = torch.LongTensor(all_objects)
all_subjects = torch.squeeze(torch.LongTensor(all_subjects))
# all_relationships = torch.LongTensor([a+1 for a in all_relationships])
all_relationships = torch.LongTensor(all_relationships)

In [9]:
model = ConvE(args = args, num_entities = entities, num_relations=4)

70555 4


In [11]:
def get_torch(a):
    oof = BCE_dictionary[(a[0],a[1])]
    return((oof.todense()))
    

In [13]:
model = ConvE(args = args, num_entities = entities, num_relations=4)
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)
for epoch in range(number_of_epochs):

        model.train() #tells pytorch you are training rather than evaluating, so dropout happens

        objects, subjects, relationships  = load_data(test_data, number_of_batches)

        for index in range(number_of_batches):

            obj = torch.LongTensor(objects[index])
            rel = torch.LongTensor(relationships[index])
            subj = torch.squeeze(torch.LongTensor(subjects[index]))

            # obj_sub = np.hstack((objects[index], relationships[index]))
            # BCE = np.apply_along_axis(get_torch, axis= 1, arr = obj_sub)
            # BCE_subj = torch.LongTensor(BCE).float()

            optimiser.zero_grad()
            pred = model.forward(e1 = obj, rel = rel)

            loss2 = torch.nn.CrossEntropyLoss()

            loss = loss2(pred, subj)
            # print(loss)
            loss.backward()
            optimiser.step()


        model.eval()

        # print(model.emb_rel(torch.LongTensor([1])))

        outputs = model.forward(e1 = all_objects, rel = all_relationships)
        _, predicted = torch.max(outputs.data, 1)

        print('number of correct answers is') 
        print((predicted == all_subjects).sum().item())


70555 4
number of correct answers is
3
number of correct answers is
15
number of correct answers is
43
number of correct answers is
61
number of correct answers is
79
number of correct answers is
92
number of correct answers is
98
number of correct answers is
100
number of correct answers is
100
number of correct answers is
100
number of correct answers is
100


KeyboardInterrupt: 

## this below model is BCE - gave up and swapped to CE 

In [11]:
model = ConvE(args = args, num_entities = entities, num_relations=4)
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)
for epoch in range(number_of_epochs):

        model.train() #tells pytorch you are training rather than evaluating, so dropout happens

        objects, subjects, relationships  = load_data(test_data, number_of_batches)

        for index in range(number_of_batches):

            obj = torch.LongTensor(objects[index])
            rel = torch.LongTensor(relationships[index])
            subj = torch.squeeze(torch.LongTensor(subjects[index]))

            obj_sub = np.hstack((objects[index], relationships[index]))
            BCE = np.apply_along_axis(get_torch, axis= 1, arr = obj_sub)
            BCE_subj = torch.LongTensor(BCE).float()

            optimiser.zero_grad()
            pred = model.forward(e1 = obj, rel = rel)

            loss = model.loss(pred, BCE_subj)
            loss.backward()
            optimiser.step()

        # print(pred[0].round().nonzero())
        print(BCE_subj[0].nonzero())


        _, index = torch.max(pred[0], 0)
        print(index)

        # print(loss)
        model.eval()

        # print(model.emb_rel(torch.LongTensor([1])))

        # outputs = model.forward(e1 = all_objects, rel = all_relationships)
        # _, predicted = torch.max(outputs.data, 1)

        # print('number of correct answers is') 
        # print((predicted == all_subjects).sum().item())



70555 4
tensor([[ 1243],
        [ 6826],
        [15588],
        [20694],
        [34531],
        [49237],
        [50161],
        [53472],
        [66330]])
tensor(68454)
tensor([[19157],
        [65626]])
tensor(63506)
tensor([[28401],
        [47833]])
tensor(11057)
tensor([[ 9733],
        [35103],
        [38727],
        [47143],
        [51620],
        [53090],
        [59967]])
tensor(19026)
tensor([[ 2822],
        [13092],
        [13495],
        [18849],
        [24212],
        [40427],
        [42836],
        [58953],
        [61703]])
tensor(19026)
tensor([[  262],
        [ 2823],
        [ 4581],
        [ 7940],
        [14179],
        [24116],
        [37764],
        [54330],
        [59125],
        [60669],
        [63720],
        [67113]])
tensor(18487)
tensor([[ 9225],
        [26620],
        [31448],
        [33073],
        [43997],
        [50237],
        [51548],
        [56387],
        [66924],
        [67993]])
tensor(2854)
tensor([[  262],
    

In [ ]:
model.eval()
outputs = model.forward(e1 = all_objects, rel = all_relationships)
_, predicted = torch.max(outputs.data, 1)
predicted

In [84]:
BCE_subj[0].nonzero()

tensor([[47309],
        [48579],
        [63957]])

In [56]:
test = outputs.round()
predicted = test.data

In [82]:
predicted[0].nonzero()

tensor([[17108],
        [32085],
        [39579],
        [48866],
        [55546],
        [69165]])

In [81]:
torch.nonzero(work[0])

tensor([[17358],
        [23807]])

In [79]:
print(sum(predicted[0]))
print(sum(work[0]))
for index, x in enumerate(predicted):
    if torch.all(torch.eq(predicted[index], work[index])) == False:
        print(work[index])

tensor(6.)
tensor(2.)
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
ten

In [63]:
fuk = np.hstack((all_objects, all_relationships))
please = np.apply_along_axis(get_torch, axis= 1, arr = fuk)
work = torch.LongTensor(please).float()

In [ ]:
outputs = model.forward(e1 = all_objects, rel = all_relationships)
_, predicted = torch.max(outputs.data, 1)
print(predicted)
print(predicted/all_subjects)

In [ ]:
# best way to test your system is to train on 100 examples and tests on the 100 same examples - if your model doesn't overfit after a bundle of epochs there is a mistake. 
# knowledge base models take a long long time to converge. They can take 100s of epochs to converge. YOU SHOULD SHUFFLE YOUR DATA EVERY TIME!!! Don't have the same stuff in each batch. 
# learn to use the data loader for shuffling. Alternatively do it yourself.  
# with BCE you can do label smoothing. 

In [ ]:
model.zero_grad()

print('emb_grad')
print(model.emb_e.weight.grad)

loss = model.loss(pred, subj)
loss.backward()

print('emb_grad')
print(model.emb_e.weight.grad)